# Estimador de precios de inmuebles en la ciudad de San Juan, Argentina.

## 1. Inicialización: librerías, configuraciones y variables globales

### Importación de Librerías

In [ ]:
!pip install joblib
!pip install lightgbm
!pip install scikit-learn
!pip install nltk
!pip install transformers
!pip install pandas
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install tensorflow
!pip install torch
!pip install tf-keras
!pip install optuna

In [ ]:
import requests
from bs4 import BeautifulSoup

import sqlite3
import re
import unicodedata
import os
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import joblib

import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('all')

from transformers import pipeline

import spacy
nlp = spacy.load('es_core_news_sm')
import optuna

## 2. Importación de Base de datos

### Buscar la base de datos

In [ ]:

conn = sqlite3.connect('inmuebles_normalizada.db')
df_all = pd.read_sql_query("SELECT * FROM inmuebles", conn)
conn.close()

In [ ]:
def limpiar_texto(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Reemplazar tildes
    text = text.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('Á', 'A').replace('É', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
    # Eliminar números y caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Eliminar espacios en blanco adicionales
    text = ' '.join(text.split())
    # Tokenizar el texto
    tokens = word_tokenize(text, language='spanish')
    # Eliminar stop words
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if not word in stop_words]
    # Lematizar las palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Unir los tokens de nuevo en una cadena
    processed_text = ' '.join(tokens)
    return processed_text

df_all['comentarios'] = df_all['comentarios'].apply(limpiar_texto)

### Realizamos un análisis de sentimientos del texto "comentarios"

In [ ]:

# Cargar el modelo de análisis de sentimientos
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Función para analizar el sentimiento de un texto
def analyze_sentiment(text):
  #Se trunca el texto a un máximo de 512 tokens para que se ajuste al modelo BERT.
  result = sentiment_pipeline(text[:512])
  return result[0]['label'], result[0]['score']

# Aplicar la función al DataFrame
df_all[['sentiment_label', 'sentiment_score']] = df_all['comentarios'].apply(lambda x: pd.Series(analyze_sentiment(x)))

# Mapear las etiquetas de sentimiento a valores numéricos
sentiment_mapping = {'1 star': 1, '2 stars': 2, '3 stars': 3, '4 stars': 4, '5 stars': 5}
df_all['des_sentimiento'] = df_all['sentiment_label'].map(sentiment_mapping)

### Realizamos una Clasificación de texto de la variable Des

In [ ]:
# Cargar el pipeline para clasificación de texto con BERT
classifier = pipeline('text-classification', model='dccuchile/bert-base-spanish-wwm-uncased')

# Función para clasificar el texto
def classify_text(text):
  result = classifier(text[:775])
  return result[0]['label'], result[0]['score']

# Aplicar la función al DataFrame
df_all[['classification_label', 'classification_score']] = df_all['comentarios'].apply(lambda x: pd.Series(classify_text(x)))
df_all['des_clasificacion'] = df_all['classification_label'].map({'LABEL_0': False, 'LABEL_1': True})


In [ ]:
df_all.head()

In [ ]:
# Eliminar las columnas especificadas
df_all = df_all.drop(columns=[
    'sentiment_label','sentiment_score', 'classification_score', 'classification_label'
])

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_all.head()


In [ ]:
df_backup = df_all.copy()

In [ ]:
df_all = df_backup.copy()

In [ ]:
# Inicializar el vectorizador TF-IDF con un prefijo para evitar duplicados

# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 1), tokenizer=word_tokenize)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_all['comentarios'])

# Obtener los nombres de features con un prefijo para evitar duplicados
feature_names = ['tfidf_' + name for name in tfidf_vectorizer.get_feature_names_out()]

# Convertir la matriz TF-IDF a un DataFrame de pandas con los nombres prefijados
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Concatenar el DataFrame de TF-IDF con el DataFrame original
df_all = pd.concat([df_all, tfidf_df], axis=1)

# Mostrar las dimensiones del DataFrame
df_all.shape


In [ ]:
df_all['precio'].head()

In [ ]:
# Función para determinar el mejor tamaño de clase
def determinar_mejor_tamanio_clase(df, columna, tamanios_clase):
    resultados = []
    for tamanio in tamanios_clase:
        # Crear clases basadas en el tamaño de clase
        bins = list(range(int(df[columna].min()), int(df[columna].max()) + tamanio, tamanio))
        df['clase'] = pd.cut(df[columna], bins=bins, include_lowest=True)
        
        # Calcular la varianza dentro de las clases
        varianza_total = df.groupby('clase')[columna].var().sum()
        resultados.append((tamanio, varianza_total))
    
    # Seleccionar el tamaño de clase con la menor varianza
    mejor_tamanio = min(resultados, key=lambda x: x[1])
    return mejor_tamanio, resultados

# Lista de tamaños de clase a evaluar
tamanios_clase = range(25000, 50000, 75000)

# Determinar el mejor tamaño de clase
mejor_tamanio, resultados = determinar_mejor_tamanio_clase(df_all, 'precio', tamanios_clase)

# Mostrar el mejor tamaño de clase
print(f"Mejor tamaño de clase: {mejor_tamanio[0]} con varianza total: {mejor_tamanio[1]}")

# Graficar los resultados
tamanios, varianzas = zip(*resultados)
plt.figure(figsize=(10, 6))
plt.plot(tamanios, varianzas, marker='o', color='blue')
plt.title('Evaluación de Tamaños de Clase', fontsize=16)
plt.xlabel('Tamaño de Clase', fontsize=12)
plt.ylabel('Varianza Total', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Eliminar filas con valores nulos en la columna 'precio'
df_all = df_all.dropna(subset=['precio'])

In [ ]:
# Agregar una nueva columna 'clase' basada en el precio
df_all['clase'] = (df_all['precio'] // 25000).astype(int)

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_all[['precio', 'clase']].head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np

# Import necessary libraries
import matplotlib.pyplot as plt

# Check if latitude and longitudee data exist in the dataframe
if 'longitude' in df_all.columns and 'latitude' in df_all.columns:
    # Create a dataframe with just latitude, longitude and class (price level)
    geo_df = df_all[['latitude', 'longitude', 'clase']].copy()
    
    # Calculate price per square meter if the data is available
    if 'superficie_total' in df_all.columns and df_all['superficie_total'].sum() > 0:
        # Recover price from class (class * 25000) and calculate price per m²
        geo_df['precio_estimado'] = geo_df['clase'] * 25000
        geo_df['precio_m2'] = geo_df['precio_estimado'] / df_all['superficie_total'].replace(0, np.nan)
        geo_df.dropna(subset=['precio_m2'], inplace=True)
    
    # Drop rows with missing coordinates
    geo_df = geo_df.dropna(subset=['latitude', 'longitude'])
    
    # Scale the geographic data for clustering
    scaler = StandardScaler()
    geo_scaled = scaler.fit_transform(geo_df[['latitude', 'longitude']])
    
    # Determine optimal number of clusters using the elbow method
    inertias = []
    k_range = range(1, 11)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(geo_scaled)
        inertias.append(kmeans.inertia_)
    
    # Plot elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertias, 'o-')
    plt.xlabel('Número de clusters')
    plt.ylabel('Inercia')
    plt.title('Método del Codo para determinar el número óptimo de clusters')
    plt.grid(True)
    plt.show()
    
    # Choose an appropriate number of clusters based on the elbow curve
    optimal_k = 5  # You might want to adjust this after seeing the elbow curve
    
    # Train the K-means model
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    geo_df['cluster'] = kmeans.fit_predict(geo_scaled)
    
    # Calculate average price and price per m² by cluster
    cluster_stats = geo_df.groupby('cluster').agg({
        'clase': 'mean',
        'precio_m2': 'mean' if 'precio_m2' in geo_df.columns else 'count'
    }).reset_index()
    
    # Visualize the clusters on a map
    plt.figure(figsize=(12, 10))
    
    # Create a colormap for the clusters
    colors = plt.cm.tab10(np.linspace(0, 1, optimal_k))
    
    # Plot each cluster with its corresponding color
    for cluster in range(optimal_k):
        cluster_points = geo_df[geo_df['cluster'] == cluster]
        plt.scatter(
            cluster_points['longitude'], 
            cluster_points['latitude'],
            c=[colors[cluster]],
            label=f'Cluster {cluster} (Precio Medio: {int(cluster_stats.loc[cluster_stats["cluster"] == cluster, "clase"].values[0] * 25000)})',
            alpha=0.7
        )
    
    # Add cluster centers
    centers = scaler.inverse_transform(kmeans.cluster_centers_)
    plt.scatter(centers[:, 1], centers[:, 0], c='black', s=200, alpha=0.5, marker='X')
    
    plt.title('Zonas de precios en San Juan basadas en ubicación geográfica')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Print statistics by cluster
    print("\nEstadísticas por cluster:")
    for i, row in cluster_stats.iterrows():
        cluster_num = row['cluster']
        avg_price = row['clase'] * 25000
        price_metric = row['precio_m2']
        
        if 'precio_m2' in geo_df.columns:
            print(f"Cluster {cluster_num}: Precio Promedio = ${int(avg_price)}, Precio/m² Promedio = ${int(price_metric)}")
        else:
            print(f"Cluster {cluster_num}: Precio Promedio = ${int(avg_price)}, Cantidad de propiedades: {int(price_metric)}")
else:
    print("No se encontraron datos de latitude y longitude en el dataset.")

In [ ]:
df_all.columns

In [ ]:
# Eliminar la variable precio
df_all = df_all.drop(columns=['precio'])

# Mostrar la forma actual del dataframe
print(f"Forma del dataframe después de eliminar 'precio': {df_all.shape}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Importar las bibliotecas necesarias

# Definir características (features) y variable objetivo (target)
X = df_all.drop(['clase', 'publicacion', 'image_blob', 'comentarios'], axis=1)
y = df_all['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Redondear predicciones al entero más cercano para obtener las clases
y_pred_classes = np.round(y_pred).astype(int)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
accuracy = np.mean(y_pred_classes == y_test)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# Visualizar la importancia de las características
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf_model.feature_importances_
})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Mostrar las 20 características más importantes
plt.figure(figsize=(12, 8))
plt.barh(feature_importance['Feature'][:20], feature_importance['Importance'][:20])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Top 20 características más importantes (Random Forest)')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Eliminar la columna 'precio'
df_all = df_all.drop(columns=['precio'])

# Definir características (features) y variable objetivo (target)
X = df_all.drop(['clase', 'publicacion', 'image_blob', 'comentarios'], axis=1)
y = df_all['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un dataset de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Definir los parámetros para el modelo
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.8,
    'num_leaves': 31,
    'max_depth': 10,
    'subsample': 0.8,
    'learning_rate': 0.1,
    'n_estimators': 100
}

# Entrenar el modelo
model = lgb.train(params,
                  train_data,
                  num_boost_round=100,
                  valid_sets=[test_data],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])  # Usar early_stopping callback

# Predecir el subconjunto de prueba
y_pred = model.predict(X_test)

# Redondear las predicciones al entero más cercano para obtener la clase
y_pred_classes = np.round(y_pred).astype(int)

# Evaluar el modelo
accuracy = np.mean(y_pred_classes == y_test)
print(f"Accuracy: {accuracy}")

In [ ]:
# Definir la función objetivo para la optimización
def objective(trial):
    # Definir los hiperparámetros a optimizar
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'num_leaves': trial.suggest_int('num_leaves', 10, 50),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 200)
    }

    # Entrenar el modelo con los hiperparámetros actuales
    model = lgb.train(params, train_data, valid_sets=[test_data])
    
    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)
    
    # Calcular el RMSE
    rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
    return rmse

# Crear un estudio para la optimización
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", study.best_params)

# Entrenar el modelo final con los mejores hiperparámetros
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'
model = lgb.train(best_params, train_data, num_boost_round=100, valid_sets=[test_data], callbacks=[lgb.early_stopping(stopping_rounds=10)])

# Predecir y evaluar el modelo optimizado
y_pred = model.predict(X_test)
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
print(f"RMSE después de la optimización bayesiana: {rmse}")

In [ ]:
# Usar los mejores hiperparámetros encontrados por Optuna
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'

# Crear un nuevo modelo con los mejores hiperparámetros
final_model = lgb.train(
    best_params,
    train_data,
    num_boost_round=best_params['n_estimators'],
    valid_sets=[test_data],
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# Predecir en el conjunto de prueba
y_pred = final_model.predict(X_test)

# Redondear las predicciones al entero más cercano para obtener la clase
y_pred_classes = np.round(y_pred).astype(int)

# Calcular la precisión (accuracy)
accuracy = np.mean(y_pred_classes == y_test)
print(f"Accuracy con los mejores hiperparámetros: {accuracy:.4f}")

# Calcular el RMSE
rmse = np.sqrt(np.mean((y_test - y_pred) ** 2))
print(f"RMSE con los mejores hiperparámetros: {rmse:.4f}")

# Mostrar la importancia de las características
importance = final_model.feature_importance()
feature_names = X_train.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Visualizar las 20 características más importantes
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'][:20], importance_df['Importance'][:20])
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Top 20 características más importantes')
plt.gca().invert_yaxis()  # Invertir el eje Y para mostrar la característica más importante arriba
plt.show()

In [ ]:

# Definir características (features) y variable objetivo (target)
X = df_all.drop(['clase', 'publicacion', 'image_blob', 'comentarios'], axis=1)
y = df_all['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear un dataset de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Definir los parámetros para el modelo
params = best_params.copy()
# Entrenar el modelo
model = lgb.train(params,
                  train_data,
                  num_boost_round=100,
                  valid_sets=[test_data],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)])  # Usar early_stopping callback

# Predecir el subconjunto de prueba
y_pred = model.predict(X_test)

# Redondear las predicciones al entero más cercano para obtener la clase
y_pred_classes = np.round(y_pred).astype(int)

# Evaluar el modelo
accuracy = np.mean(y_pred_classes == y_test)
print(f"Accuracy: {accuracy}")